In [ ]:
import numpy as np

vec = np.load("../data/hog/0.npy")
print(vec.shape)
print(vec[:20])


(8100,)
[0.29169273 0.17284098 0.07630757 0.10154331 0.15816405 0.02941237
 0.15900024 0.05533042 0.21555001 0.06538779 0.11610029 0.02647543
 0.1133854  0.32852079 0.13308106 0.1288689  0.05517796 0.04610085
 0.09782295 0.04713698]


In [26]:
tokens = [126, 217, 124, 183, 192, 126, 217, 123, 523, 124, 123, 126, 217]
counter = dict()
for i in range(len(tokens)):
    if i+1 < len(tokens) and [tokens[i], tokens[i+1]] in tokens:
        counter[tokens[i], tokens[i+1]] += 1
        
        
counter

{}

In [34]:
tokens = [126, 217, 124, 183, 192, 126, 217, 123, 523, 124, 123, 126, 217]
a = []
for n1, n2 in zip(tokens, tokens[1:]):
    a.append([n1,n2])
    
L = list(set(map(tuple, a)))
dict = {item: list(map(tuple, a)).count(item) for item in list(map(tuple, a))}
dict

{(126, 217): 3,
 (217, 124): 1,
 (124, 183): 1,
 (183, 192): 1,
 (192, 126): 1,
 (217, 123): 1,
 (123, 523): 1,
 (523, 124): 1,
 (124, 123): 1,
 (123, 126): 1}

In [1]:
text = """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""
# tokens = text.encode("utf-8") # raw bytes
# tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
# print(tokens)

In [2]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    result = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            result.append(idx)
            i += 2
        else:
            result.append(ids[i])
            i += 1
    return result

vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (101, 114) into a new token 260
merging (99, 111) into a new token 261
merging (116, 32) into a new token 262
merging (226, 128) into a new token 263
merging (44, 32) into a new token 264
merging (97, 110) into a new token 265
merging (111, 114) into a new token 266
merging (100, 32) into a new token 267
merging (97, 114) into a new token 268
merging (101, 110) into a new token 269
merging (257, 103) into a new token 270
merging (261, 100) into a new token 271
merging (121, 32) into a new token 272
merging (46, 32) into a new token 273
merging (97, 108) into a new token 274
merging (259, 256) into a new token 275


In [36]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
    
def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text
print(decode(ids))    

A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/r

In [4]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >=2:
        stats = get_stats(tokens)
        pair = min(stats, key = lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("hello world!"))

[104, 101, 108, 108, 111, 32, 119, 266, 108, 100, 33]


In [5]:
text2 = decode(encode(text))
print(text2 == text)

True


In [28]:
import pandas as pd
from collections import Counter

df = pd.read_csv('../data/raw/train.csv')
captions = df['caption'].astype(str).tolist()


In [38]:
with open("../data/raw/train_captions.txt", "w", encoding="utf-8") as f:
    for cap in captions:
        f.write(cap.strip() + "\n")

In [51]:
words = [w for cap in captions for w in cap.split()]
word_counts = Counter(words)

In [52]:
len(word_counts)

37334

In [21]:
import sentencepiece as spm
import os

options = dict(
    # Input Specs:
    input = '../data/raw/train_captions.txt',
    input_format = 'text',
    # Output Specs:
    model_prefix = 'caption_bpe',
    # Algorithm Specs:
    model_type = 'bpe',
    vocab_size = '6000',
    # Rare Word Treatment
    character_coverage = 0.99995,
    byte_fallback = True,
    # Normalization:
    normalization_rule_name = 'identity',
    remove_extra_whitespaces = False,
    input_sentence_size = 200000000,
    max_sentence_length = 4192,
    seed_sentencepiece_size = 1000000,
    shuffle_input_sentence = True,
    # Merge Rules:
    split_digits = True,
    split_by_unicode_script = True,
    split_by_whitespace = True,
    split_by_number = True,
    max_sentencepiece_length = 16,
    add_dummy_prefix = True,
    allow_whitespace_only_pieces = True,
    # Special Tokens:
    unk_id = 0,
    bos_id = 1,
    eos_id = 2,
    pad_id = 3,
    # Systems:
    num_threads = os.cpu_count() # Use all system resources
)
spm.SentencePieceTrainer.train(**options)

In [22]:
sp = spm.SentencePieceProcessor()
sp.load('caption_bpe.model')

True

In [ ]:
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.GetPieceSize())]
vocab

In [38]:
len(vocab)

6000

In [36]:
captions_ids = sp.EncodeAsIds(captions[:5])
print(captions_ids)

[[333, 341, 385, 260, 1883, 5873, 1948, 2432, 1455, 260, 818, 595, 260, 4918, 5861, 290, 1072, 4525, 281, 3210, 702, 962, 5863], [2109, 2886, 338, 309, 5885, 5605, 595, 888, 2084, 533, 291, 394, 5309, 5861, 366, 5863, 5871, 502, 290, 1670, 311, 1991, 3551, 281, 686, 2147, 5863], [496, 2529, 4353, 327, 260, 1819, 311, 1141, 5861, 1276, 290, 473, 1411, 5861, 285, 260, 1110, 1305, 759, 5863], [278, 482, 2767, 465, 300, 383, 4239, 3805, 507, 1654, 1551, 5841, 5891, 5883, 5886, 5902, 5861, 290, 954, 1858, 478, 301, 670, 2038, 1230, 1869, 5863], [278, 836, 5873, 480, 311, 1010, 776, 1459, 1331, 1098, 5861, 438, 982, 2410, 5633, 281, 3477, 1003, 5863]]


In [39]:
display(captions[:5])
sp.DecodeIds(captions_ids)

['The image features a comic-style panel depicting a scene from a story, with dialogue and narration above.',
 'Colorful postcard featuring "Greetings from Cherry Grove Beach, S.C." with images of beach scenes and vibrant lettering.',
 'Two vending machines display a variety of drinks, illuminated with colorful lights, in a train station setting.',
 'A man speaks at the eGovernment Conference 2013, with multiple logos displayed on computer monitors behind him.',
 'A close-up of several silver coins stacked together, showcasing their shiny surfaces and engraved designs.']

['The image features a comic-style panel depicting a scene from a story, with dialogue and narration above.',
 'Colorful postcard featuring "Greetings from Cherry Grove Beach, S.C." with images of beach scenes and vibrant lettering.',
 'Two vending machines display a variety of drinks, illuminated with colorful lights, in a train station setting.',
 'A man speaks at the eGovernment Conference 2013, with multiple logos displayed on computer monitors behind him.',
 'A close-up of several silver coins stacked together, showcasing their shiny surfaces and engraved designs.']

In [40]:
print([sp.IdToPiece(idx) for idx in captions_ids])

[['▁The', '▁image', '▁features', '▁a', '▁comic', '-', 'style', '▁panel', '▁depicting', '▁a', '▁scene', '▁from', '▁a', '▁story', ',', '▁with', '▁dial', 'ogue', '▁and', '▁nar', 'ration', '▁above', '.'], ['▁Colorful', '▁postcard', '▁featuring', '▁"', 'G', 'reetings', '▁from', '▁Ch', 'erry', '▁G', 'ro', 've', '▁Beach', ',', '▁S', '.', 'C', '."', '▁with', '▁images', '▁of', '▁beach', '▁scenes', '▁and', '▁vibrant', '▁lettering', '.'], ['▁Two', '▁vending', '▁machines', '▁display', '▁a', '▁variety', '▁of', '▁drinks', ',', '▁illuminated', '▁with', '▁colorful', '▁lights', ',', '▁in', '▁a', '▁train', '▁station', '▁setting', '.'], ['▁A', '▁man', '▁speaks', '▁at', '▁the', '▁e', 'Go', 'vern', 'ment', '▁Con', 'ference', '▁', '2', '0', '1', '3', ',', '▁with', '▁multiple', '▁logos', '▁displayed', '▁on', '▁computer', '▁monitors', '▁behind', '▁him', '.'], ['▁A', '▁close', '-', 'up', '▁of', '▁several', '▁silver', '▁coins', '▁stacked', '▁together', ',', '▁showcasing', '▁their', '▁shiny', '▁surfaces', '▁and'

In [49]:
ids = sp.EncodeAsIds("a dog running across the field")
tokens = sp.EncodeAsPieces("a dog running across the field")

print(ids)
print(tokens)

[260, 2575, 2438, 2613, 300, 571]
['▁a', '▁dog', '▁running', '▁across', '▁the', '▁field']


In [50]:
input_ids = [1] + sp.EncodeAsIds("a dog running across the field")
target_ids = sp.EncodeAsIds("a dog running across the field") + [2]

max_len = 30
input_ids_padded = input_ids + [3]*(max_len - len(input_ids))
target_ids_padded = target_ids + [3]*(max_len - len(target_ids))

print(input_ids_padded)
print(target_ids_padded)

[1, 260, 2575, 2438, 2613, 300, 571, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[260, 2575, 2438, 2613, 300, 571, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [18]:
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize
import numpy as np
from pathlib import Path
from tqdm import tqdm

image_path = '../data/raw/train/train/1548.jpg'
target_size = (256, 256)

hog_params ={
           'orientations': 9,
           'pixels_per_cell': (8, 8),
           'cells_per_block': (2, 2),
           'block_norm': 'L2-Hys',
           'visualize': False,
           'feature_vector': True
       }

image = imread(str(image_path))
if image.ndim == 3 and image.shape[-1] == 4:
    # RGBA → RGB → Gray
    image = image[..., :3]
    gray = rgb2gray(image)
elif image.ndim == 3 and image.shape[-1] == 3:
    # RGB → Gray
    gray = rgb2gray(image)
elif image.ndim == 2:  
    # Already grayscale
    gray = image
else:
    raise ValueError(f"Unexpected image shape {image.shape}")
    
gray = resize(gray, target_size, anti_aliasing=True)
hog_vec = hog(gray, **hog_params)
hog_vec = hog_vec.astype(np.float32)

In [19]:
print(hog_vec, hog_vec.shape)

[0.06658278 0.11205684 0.13339299 ... 0.06305587 0.18832268 0.02542144] (34596,)


In [20]:
h, w = target_size
orientations = hog_params['orientations']
pixels_per_cell = hog_params['pixels_per_cell']
cells_per_block = hog_params['cells_per_block']

cells_h = h // pixels_per_cell[0]
cells_w = w // pixels_per_cell[1]
blocks_h = cells_h - cells_per_block[0] + 1
blocks_w = cells_w - cells_per_block[1] + 1

dimension =  blocks_h * blocks_w * cells_per_block[0] * cells_per_block[1] * orientations
print(dimension)

34596


In [2]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))


In [3]:
from src.tokenizer.tokenizer import BPETokenizer
tokenizer = BPETokenizer("caption_bpe.model")

In [4]:
tokenizer.train('../data/raw/train_captions.txt')

In [34]:
test_token_ids = tokenizer.encode(text[:250])

In [ ]:
test_token_ids

In [36]:
test_decoded_text = tokenizer.decode(test_token_ids)

In [38]:
test_decoded_text == text[:250]

True

In [54]:
test_input_tokens, test_target_tokens = tokenizer.get_input_and_target(test_token_ids, 180)

In [55]:
print(test_input_tokens, test_target_tokens)

[1, 278, 2296, 5854, 781, 985, 5918, 5848, 4404, 5697, 506, 358, 1595, 297, 4508, 4064, 5841, 5902, 5861, 5841, 5891, 5883, 5886, 5904, 5841, 198, 187, 373, 379, 265, 5841, 198, 187, 5841, 5891, 5891, 2749, 873, 5841, 5841, 243, 192, 185, 243, 193, 146, 243, 193, 141, 243, 193, 135, 243, 193, 147, 243, 193, 136, 243, 193, 137, 5910, 5841, 244, 163, 137, 168, 244, 163, 137, 161, 244, 163, 137, 156, 244, 163, 137, 150, 244, 163, 137, 162, 244, 163, 137, 151, 244, 163, 137, 152, 230, 132, 193, 5841, 244, 163, 139, 190, 230, 132, 144, 244, 163, 139, 183, 230, 132, 144, 244, 163, 139, 178, 230, 132, 144, 244, 163, 139, 172, 230, 132, 144, 244, 163, 139, 184, 230, 132, 144, 244, 163, 139, 173, 230, 132, 144, 244, 163, 139, 174, 5910, 5841, 244, 163, 156, 136, 333, 317, 856, 1299, 2792, 3497, 274, 370, 281, 260, 1433, 1803, 300, 811, 279, 356, 311, 2360, 5858, 332, 5111, 5860, 398, 5863, 2193, 2137, 2703, 637, 916, 293, 2114, 5846, 358] [278, 2296, 5854, 781, 985, 5918, 5848, 4404, 5697, 506,

In [57]:
tokenizer.decode(test_input_tokens)

'A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to'